In [85]:
import pickle
import pandas as pd

In [86]:
year=2050
nuc="nuclear_plus"
bati_hyp="ref"
reindus="reindus"
with open("Results_"+str(year)+"_"+nuc+"_"+bati_hyp+"_"+reindus+".pickle", 'rb') as f:
    Variables=pickle.load(f)

In [87]:
print(Variables)

{'power_Dvar':                       TIME    TECHNOLOGIES    power_Dvar
0      2019-12-13 14:00:00      HydroRiver   8188.273572
1      2019-12-13 14:00:00    WindOffShore  65571.622614
2      2019-12-13 14:00:00         Biomass      0.000000
3      2019-12-13 14:00:00   NewHydroRiver    890.836653
4      2019-12-13 14:00:00     curtailment      0.000000
...                    ...             ...           ...
113875 2019-06-21 15:00:00     WindOnShore   2171.900000
113876 2019-06-21 15:00:00             TAC      0.000000
113877 2019-06-21 15:00:00  HydroReservoir      0.000000
113878 2019-06-21 15:00:00             CCG      0.000000
113879 2019-06-21 15:00:00            Coal      0.000000

[113880 rows x 3 columns], 'energy_Pvar':                     TIME    energy_Pvar
0    2019-12-13 14:00:00  137163.059581
1    2019-11-05 14:00:00  115461.868517
2    2019-04-24 16:00:00  121589.253296
3    2019-02-07 09:00:00  100026.902907
4    2019-01-08 04:00:00   94912.002112
...               

# 1. Capacity 

In [88]:
# Production means capacity (MW)
Variables['capacity_Dvar']

,TECHNOLOGIES,capacity_Dvar
0,HydroRiver,11030.000000
1,WindOffShore,67641.451015
2,Biomass,2031.000000
3,NewHydroRiver,1200.000000
4,curtailment,0.000000
5,NewNuke,25080.000000
6,OldNuke,15520.000000
7,Solar,150000.000000
8,WindOnShore,37000.000000
9,TAC,0.000000


In [89]:
# Storage means capacity (MW)
Variables['total_consumption_Pvar'].max()

TIME                      2019-12-31 23:00:00
total_consumption_Pvar          155534.579971
dtype: object

In [90]:
Variables['energy_Pvar'].max()

TIME           2019-12-31 23:00:00
energy_Pvar          171065.850037
dtype: object

In [91]:
df=Variables['power_Dvar']
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(df[df.TIME==pd.Timestamp('2019-12-31 23:00:00')]['power_Dvar'].sum())
df2=Variables['storageOut_Pvar']
print(df2[df2.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df3=Variables['storageIn_Pvar']
print(df3[df3.TIME==pd.Timestamp('2019-12-31 23:00:00')])
df4=Variables['stockLevel_Pvar']
print(df4[df4.TIME==pd.Timestamp('2019-12-31 23:00:00')])
print(Variables['stockLevel_ini_Dvar'])

                     TIME    TECHNOLOGIES    power_Dvar
36088 2019-12-31 23:00:00      HydroRiver   8233.891861
36089 2019-12-31 23:00:00    WindOffShore  14630.845855
36090 2019-12-31 23:00:00         Biomass   2031.000000
36091 2019-12-31 23:00:00   NewHydroRiver    895.799659
36092 2019-12-31 23:00:00     curtailment      0.000000
36093 2019-12-31 23:00:00         NewNuke  21710.000000
36094 2019-12-31 23:00:00         OldNuke  15520.000000
36095 2019-12-31 23:00:00           Solar      0.000000
36096 2019-12-31 23:00:00     WindOnShore   4432.600000
36097 2019-12-31 23:00:00             TAC      0.000000
36098 2019-12-31 23:00:00  HydroReservoir   8785.000000
36099 2019-12-31 23:00:00             CCG      0.000000
36100 2019-12-31 23:00:00            Coal      0.000000
76239.13737415877
                    TIME      STOCK_TECHNO  storageOut_Pvar
8328 2019-12-31 23:00:00      HydroStorage              0.0
8329 2019-12-31 23:00:00           Battery              0.0
8330 2019-12-31 23

In [92]:
df=Variables['power_Dvar']
df2=Variables['storageOut_Pvar']
df3=Variables['storageIn_Pvar']
df=df.groupby('TIME')['power_Dvar'].sum()
df2=df2.groupby('TIME')['storageOut_Pvar'].sum()
df3=df3.groupby('TIME')['storageIn_Pvar'].sum()
df_prod=pd.DataFrame()
df_prod['Prod']=df+df2+df3
#df_prod
#df_prod=df_prod.set_index('TIME')

df_diff=pd.DataFrame()
df_diff['diff']=df_prod['Prod']-Variables['total_consumption_Pvar'].set_index('TIME')['total_consumption_Pvar']
print(df_diff['diff'].max())
N_err=0
for i in df_diff.index:
    if df_diff.loc[i,'diff']!=0:
        N_err+=1
print(N_err)

72294.1065228837
7698
